# RFM Analysis Notebook
This notebook:
1) Reads raw Olist data (orders, order_items, customers, products, category translations).
2) Merges everything into a single DataFrame, focusing on customers' orders and item details.
3) Computes RFM: Recency, Frequency, Monetary for each customer.
4) Integrates product category data so we can analyze spending by category as well.
5) Saves a pickled DataFrame (`rfm_data.pkl`) for use in a Streamlit dashboard.

## 1. Setup and libraries

In [1]:
# 1. import all required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from datetime import timedelta

# For inline plots
%matplotlib inline
sns.set_style("whitegrid")

## 2. Reading Olist Data

In [4]:
# 2.1 Read CSVs
orders_df = pd.read_csv(
    "../data/olist_orders_dataset.csv", 
    parse_dates=[
        "order_purchase_timestamp",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ]
)
order_items_df = pd.read_csv("../data/olist_order_items_dataset.csv")
customers_df = pd.read_csv("../data/olist_customers_dataset.csv")
products_df = pd.read_csv("../data/olist_products_dataset.csv")
categories_df= pd.read_csv("../data/product_category_name_translation.csv")

print("orders_df shape:", orders_df.shape)
print("order_items_df shape:", order_items_df.shape)
print("customers_df shape:", customers_df.shape)
print("products_df shape:", products_df.shape)
print("category_df shape:", category_translation_df.shape)

orders_df shape: (99441, 8)
order_items_df shape: (112650, 7)
customers_df shape: (99441, 5)
products_df shape: (32951, 9)
category_df shape: (71, 2)


## 3. Merges & Data Cleaning

In [5]:
# 3.1 Merge Product Data with Category Translations
products_merged = pd.merge(
    products_df,
    categories_df,
    on="product_category_name",
    how="left"
).rename(columns={
    "product_category_name_english": "product_category_name_en"
})

print("products_merged:", products_merged.shape)
products_merged.head(3)

products_merged: (32951, 10)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_en
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure


In [6]:
# 3.2 Merge order_items with orders
order_item_orders = pd.merge(
    order_items_df,
    orders_df,
    on="order_id",
    how="left"
)

# 3.3 Merge that with products_merged
df_item = pd.merge(
    order_item_orders,
    products_merged,
    on="product_id",
    how="left"
)

# 3.4 Merge with customers (for customer_unique_id)
df_item = pd.merge(
    df_item,
    customers_df[["customer_id","customer_unique_id"]],
    on="customer_id",
    how="left"
)

print("df_item shape:", df_item.shape)
df_item.head(3)

df_item shape: (112650, 24)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_en,customer_unique_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,871766c5855e863f6eccc05f988b23cb
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,eb28e67c4c0b83846050ddfb8a35d051
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,3818d81c6709e39d06b2738a8d3a2474


## 4. Basic Cleaning & Preparation

In [7]:
# Convert main timestamp to date for later usage
df_item["order_purchase_date"] = df_item["order_purchase_timestamp"].dt.date

# Create a total price column (price + freight, optional)
df_item["total_price"] = df_item["price"] + df_item["freight_value"]

## 5. Build a Customer-Level Table for RFM

In [9]:
# 5.1 Group by customer_unique_id, compute:

customer_group = df_item.groupby("customer_unique_id", as_index=False).agg({
    "order_purchase_date": ["max","min","count"],
    "total_price": "sum"
})

# Flatten multi-level columns
customer_group.columns = [
    "customer_unique_id",
    "last_purchase_date",
    "first_purchase_date",
    "order_count",
    "monetary_total"
]

print("customer_group shape:", customer_group.shape)
customer_group.head(3)

customer_group shape: (95420, 5)


,customer_unique_id,last_purchase_date,first_purchase_date,order_count,monetary_total
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,2018-05-10,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,2018-05-07,1,27.19
2,0000f46a3911fa3c0805444483337064,2017-03-10,2017-03-10,1,86.22


In [10]:
# 5.2 Define Recency, Frequency, Monetary
# define an 'analysis_date' as 1 day after the last purchase in the dataset
analysis_date = pd.to_datetime(customer_group["last_purchase_date"].max()) + pd.Timedelta(days=1)
print("analysis_date:", analysis_date)

# Convert last_purchase_date to datetime
customer_group["last_purchase_date"] = pd.to_datetime(customer_group["last_purchase_date"])

# Recency (days since last purchase)
customer_group["recency"] = (analysis_date - customer_group["last_purchase_date"]).dt.days

# Frequency
customer_group["frequency"] = customer_group["order_count"]

# Monetary
customer_group["monetary"] = customer_group["monetary_total"]

# Keep final rfm columns
rfm_df = customer_group[[
    "customer_unique_id",
    "last_purchase_date",
    "recency",
    "frequency",
    "monetary"
]].copy()

print("rfm_df shape:", rfm_df.shape)
rfm_df.head(3)


analysis_date: 2018-09-04 00:00:00
rfm_df shape: (95420, 5)


,customer_unique_id,last_purchase_date,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,117,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,120,1,27.19
2,0000f46a3911fa3c0805444483337064,2017-03-10,543,1,86.22


`df_item` is a **row-level** item DataFrame, while `rfm_df` is the **customer-level** RFM DataFrame.

In [11]:
df_item.to_pickle("df_item.pkl")
print("Saved df_item.pkl, which is a row-level item Dataframe")

Saved df_item.pkl, which is a row-level item Dataframe


## 6. K-means Clustering

In [12]:
# Simple K-Means with k=4 as an example
from sklearn.preprocessing import MinMaxScaler

X = rfm_df[["recency","frequency","monetary"]].values

# Scale data 
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)
rfm_df["cluster_k4"] = kmeans.labels_

rfm_df.head(5)

C:\Users\kealankuar.wh\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\kealankuar.wh\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


,customer_unique_id,last_purchase_date,recency,frequency,monetary,cluster_k4
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,117,1,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,120,1,27.19,1
2,0000f46a3911fa3c0805444483337064,2017-03-10,543,1,86.22,3
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12,327,1,43.62,0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14,294,1,196.89,0


In [14]:
rfm_df.head(3)

,customer_unique_id,last_purchase_date,recency,frequency,monetary,cluster_k4
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,117,1,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,120,1,27.19,1
2,0000f46a3911fa3c0805444483337064,2017-03-10,543,1,86.22,3


In [16]:
rfm_df = rfm_df[[
    "customer_unique_id",
    "last_purchase_date",
    "recency", "frequency", "monetary",
    "cluster_k4"   
]].copy()

rfm_df.head(3)

,customer_unique_id,last_purchase_date,recency,frequency,monetary,cluster_k4
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,117,1,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,120,1,27.19,1
2,0000f46a3911fa3c0805444483337064,2017-03-10,543,1,86.22,3


In [18]:
rfm_df.to_pickle("rfm_df.pkl")
print("Saved rfm_cluster_data.pkl. Contains recency, frequency, monetary, last_purchase_date, and cluster_k4.")


Saved rfm_cluster_data.pkl. Contains recency, frequency, monetary, last_purchase_date, and cluster_k4.
